# TODO
 
* ## https://medium.com/analytics-vidhya/deploy-huggingface-s-bert-to-production-with-pytorch-serve-27b068026d18 

* ## https://github.com/pytorch/serve/tree/master/examples/Huggingface_Transformers

# Deploying our BERT PyTorch Model as REST EndPoint

In [9]:
!pip install -q transformers==2.8.0
!pip install -q torch==1.5.0 --upgrade --ignore-installed

In [34]:
!pip install torchserve torch-model-archiver

In [2]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Clone the TorchServe repository and install torch-model-archiver

You'll use `torch-model-archiver` to create a model archive file (.mar). The .mar model archive file contains model checkpoints along with it’s `state_dict` (dictionary object that maps each layer to its parameter tensor).

In [1]:
!git clone https://github.com/pytorch/serve.git
!pip install serve/model-archiver/

Cloning into 'serve'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 9065 (delta 103), reused 113 (delta 32), pack-reused 8799
Receiving objects: 100% (9065/9065), 35.56 MiB | 102.30 MiB/s, done.
Resolving deltas: 100% (4853/4853), done.
Processing ./serve/model-archiver
  Using cached enum_compat-0.0.3-py3-none-any.whl (1.3 kB)
  Created wheel for torch-model-archiver: filename=torch_model_archiver-0.1.1b20200704-py3-none-any.whl size=15785 sha256=c1c0f4d303de415c8f08e76ad3c85bda60a71d6cfe68b939225c6d2d5c1b67cd
  Stored in directory: /home/ec2-user/.cache/pip/wheels/30/46/a4/0551adcd75fd74e180ed4b8c09ffca21c3f1fb910a31675843
Successfully built torch-model-archiver


# Retrieve PyTorch Models

In [6]:
%store -r s3_pytorch_model_path

In [7]:
print(s3_pytorch_model_path)

s3://sagemaker-us-east-1-806570384721/models/pytorch/pytorch_model.pt


In [10]:
%store -r s3_transformer_pytorch_model_path

In [11]:
print(s3_transformer_pytorch_model_path)

s3://sagemaker-us-east-1-806570384721/models/transformer-pytorch/


In [12]:
!aws s3 cp --recursive $s3_transformer_pytorch_model_path ./Transformer_model/

download: s3://sagemaker-us-east-1-806570384721/models/transformer-pytorch/config.json to Transformer_model/config.json
download: s3://sagemaker-us-east-1-806570384721/models/transformer-pytorch/pytorch_model.bin to Transformer_model/pytorch_model.bin


# Create TorchServe Model Archive File

Once, setup_config.json, sample_text.txt and index_to_name.json are set properly, we can go ahead and package the model and start serving it. The artifacts realted to each operation mode (such as sample_text.txt, index_to_name.json) can be place in their respective folder. 

In [ ]:
#!torch-model-archiver 
#    --model-name "bert" \
#    --version 1.0 \
#    --serialized-file ./bert_model/pytorch_model.bin \
#    --extra-files "./bert_model/config.json" \
#    --handler "./transformers_classifier_torchserve_handler.py"

In [17]:
!torch-model-archiver \
    --model-name DistilBertForSequenceClassification \
    --version 1.0 \
    --serialized-file Transformer_model/pytorch_model.bin \
    --handler ./src_torchserve/Transformer_handler_generalized.py \
    --extra-files "./Transformer_model/config.json,./src_torchserve/setup_config.json,./src_torchserve/Seq_classification_artifacts/index_to_name.json"

In [18]:
!ls ./*.mar

./DistilBertForSequenceClassification.mar


# Registering the Model on TorchServe and Running Inference

To register the model on TorchServe using the above model archive file, we run the following commands:

In [23]:
!mkdir ./model_store
!mv ./DistilBertForSequenceClassification.mar ./model_store/

mkdir: cannot create directory ‘./model_store’: File exists


In [40]:
!torchserve \
--start \
--model-store model_store \
--models distilbert-pytorch=DistilBertForSequenceClassification.mar

Removing orphan pid file.


## To run the inference using our registered model, open a new terminal and run: 

In [42]:
!curl -X POST http://127.0.0.1:8080/predictions/distilbert-pytorch -T ./src_torchserve/Seq_classification_artifacts/sample_text.txt

curl: (7) Failed to connect to 127.0.0.1 port 8080: Connection refused


# Prepare the Model for SageMaker Deployment

## Upload .mar to S3

In [ ]:
torchserve_model_name = 'DistilBertForSequenceClassification.mar'

In [ ]:
s3_torchserve_mar = 's3://{}/models/torchserve/{}'.format(bucket, torchserve_model_name)
print(s3_torchserve_mar)

In [ ]:
!aws s3 cp $s3_torchserve_mar $s3_torchserve_mar

# Store Endpoint Name for Next Notebook(s)

In [ ]:
%store endpoint_name

# _Wait Until the ^^ Endpoint ^^ is Deployed_

In [ ]:
client = boto3.client('sagemaker')
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

# Simulate a Prediction from an Application

In [ ]:
class RequestHandler(object):
    import json
    
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __call__(self, instances):
        transformed_instances = []

        for instance in instances:
            encode_plus_tokens = tokenizer.encode_plus(instance,
                                                       pad_to_max_length=True,
                                                       max_length=self.max_seq_length)

            input_ids = encode_plus_tokens['input_ids']
            input_mask = encode_plus_tokens['attention_mask']
            segment_ids = [0] * self.max_seq_length

            transformed_instance = {"input_ids": input_ids, 
                                    "input_mask": input_mask, 
                                    "segment_ids": segment_ids}

            transformed_instances.append(transformed_instance)

        transformed_data = {"instances": transformed_instances}

        return json.dumps(transformed_data)

In [ ]:
class ResponseHandler(object):
    import json
    import tensorflow as tf
    
    def __init__(self, classes):
        self.classes = classes
    
    def __call__(self, response, accept_header):
        import tensorflow as tf

        response_body = response.read().decode('utf-8')

        response_json = json.loads(response_body)

        log_probabilities = response_json["predictions"]

        predicted_classes = []

        # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
        for log_probability in log_probabilities:
            softmax = tf.nn.softmax(log_probability)    
            predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
            predicted_class = self.classes[predicted_class_idx]
            predicted_classes.append(predicted_class)

        return predicted_classes

In [ ]:
import json
from sagemaker.tensorflow.serving import Predictor
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

request_handler = RequestHandler(tokenizer=tokenizer,
                                 max_seq_length=128)

response_handler = ResponseHandler(classes=[1, 2, 3, 4, 5])

predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sess,
                      serializer=request_handler,
                      deserializer=response_handler,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

In [ ]:
import tensorflow as tf
import json
    
reviews = ["This is great!", 
           "This is terrible."]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

# Simulate a (Mini-)Load Test

In [ ]:
def predict(idx):
    reviews = ["This is great!", 
               "This is terrible."]

    predicted_classes = predictor.predict(reviews)

    return predicted_classes

In [ ]:
import functools
import multiprocessing

num_cpus = multiprocessing.cpu_count()

p = multiprocessing.Pool(num_cpus)

In [ ]:
%%time

results = p.map(predict, range(1,100))

# Verify that Elastic Inference is working
https://docs.aws.amazon.com/sagemaker/latest/dg/ei.html

_Note:  This may take 10-15 minutes for the metrics to appear._

In [ ]:
!aws cloudwatch list-metrics --namespace " AWS/ElasticInference "

# Optimize Cost with TensorFlow and Elastic Inference
https://aws.amazon.com/blogs/machine-learning/optimizing-costs-in-amazon-elastic-inference-with-amazon-tensorflow/

# Using API Gateway with SageMaker Endpoints

https://aws.amazon.com/blogs/machine-learning/creating-a-machine-learning-powered-rest-api-with-amazon-api-gateway-mapping-templates-and-amazon-sagemaker/